# Description

From the intersections groups (visualized in the UpSet plot), it samples from the "Agreements", that is, only intersection groups where there is agreement between methods (clustermatch, pearson and spearman).

# Notebook disabled

In [1]:
display("This notebook is disabled, it won't run with papermill")

import sys

sys.exit(0)

"This notebook is disabled, it won't run with papermill"

SystemExit: 0

/opt/conda/envs/clustermatch_gene_expr/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Modules

In [ ]:
import pandas as pd
import numpy as np

from clustermatch import conf

# Settings

In [ ]:
DATASET_CONFIG = conf.GTEX
GTEX_TISSUE = "whole_blood"
GENE_SEL_STRATEGY = "var_pc_log2"

In [ ]:
# amount of gene pairs to sample
SAMPLE_SIZE = 33000

# number of samples to take
N_SAMPLES = 1

# Paths

In [ ]:
INPUT_GENE_PAIRS_INTERSECTIONS_FILE = (
    DATASET_CONFIG["GENE_PAIR_INTERSECTIONS"]
    / f"gene_pair_intersections-gtex_v8-{GTEX_TISSUE}-{GENE_SEL_STRATEGY}.pkl"
)
display(INPUT_GENE_PAIRS_INTERSECTIONS_FILE)

assert INPUT_GENE_PAIRS_INTERSECTIONS_FILE.exists()

In [ ]:
OUTPUT_DIR = INPUT_GENE_PAIRS_INTERSECTIONS_FILE.parent / "samples"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
display(OUTPUT_DIR)

In [ ]:
OUTPUT_FILE_TEMPLATE = str(
    OUTPUT_DIR
    / (
        f"{INPUT_GENE_PAIRS_INTERSECTIONS_FILE.stem}-agreements_sample_"
        + "{sample_id}"
        + ".pkl"
    )
)

display(OUTPUT_FILE_TEMPLATE)

# Data

## Gene pairs intersection data

In [ ]:
def is_intersection_column(column_name):
    return " (high)" in column_name or " (low)" in column_name

In [ ]:
gene_pairs_intersections = pd.read_pickle(INPUT_GENE_PAIRS_INTERSECTIONS_FILE)

In [ ]:
gene_pairs_intersections = gene_pairs_intersections[
    [c for c in gene_pairs_intersections.columns if is_intersection_column(c)]
]

In [ ]:
gene_pairs_intersections.shape

In [ ]:
gene_pairs_intersections.head()

## Gene pairs intersection - gene pairs

In [ ]:
# gene_pairs_df = gene_pairs_intersections.rename_axis(("gene0", "gene1")).index.to_frame(index=False)

In [ ]:
# gene_pairs_df.shape

In [ ]:
# gene_pairs_df.head()

# Functions

In [ ]:
# df_r_data = df_plot

In [ ]:
# df_r_data.head()

In [ ]:
# df_r_data_boolean_cols = set(
#     [x for x in df_r_data.columns if " (high)" in x or " (low)" in x]
# )

In [ ]:
# df_r_data_boolean_cols

In [ ]:
def get_gene_pairs(gene_pairs_intersections, query_set):
    """
    FIXME: UPDATE

    It queries the given dataframe with the intersections of different groups (i.e.,
    clustermatch high, pearson low, etc) given a query set. It returns a slice of
    the dataframe according to the query set provided.

    Args:
        gene_pairs_intersections: a dataframe with gene pairs in rows and intersection
            groups as columns (which are boolean). No other columns are allowed.
        query_set: a tuple with strings that specifies a query. For example
            ("Clustermatch (high)", "Pearson (low") would select all gene pairs
            for which clustermatch is high and pearson is low.

    Returns:
        A slice of gene pairs in the input dataframe where the conditions specified in query_set
        apply.
    """
    assert all([x in gene_pairs_intersections.columns for x in query_set])

    query = np.concatenate(
        [
            # columns that have to be true
            np.concatenate(
                [
                    gene_pairs_intersections[c].to_numpy().reshape(-1, 1)
                    for c in query_set
                ],
                axis=1,
            )
            .all(axis=1)
            .reshape(-1, 1),
            # rest of the columns, that have to be false
            np.concatenate(
                [
                    ~gene_pairs_intersections[c].to_numpy().reshape(-1, 1)
                    for c in gene_pairs_intersections.columns
                    if c not in query_set
                ],
                axis=1,
            )
            .all(axis=1)
            .reshape(-1, 1),
        ],
        axis=1,
    ).all(axis=1)

    _tmp_df = gene_pairs_intersections[query]

    # _tmp_df = _tmp_df[
    #     [x for x in _tmp_df.columns if "(high)" not in x and "(low)" not in x]
    # ]

    return _tmp_df.rename_axis(("gene0", "gene1")).index.to_frame(index=False)

In [ ]:
_tmp = get_gene_pairs(
    gene_pairs_intersections,
    {
        "Clustermatch (high)",
        "Pearson (high)",
        "Spearman (high)",
    },
)

display(_tmp.dtypes)
display(_tmp.shape)
display(_tmp.head())

assert _tmp.shape[0] > int(3.12e6)

# Get intersections dataframe

## Agreements

In [ ]:
_queries = [
    [
        "Clustermatch (high)",
        "Pearson (high)",
        "Spearman (high)",
        # "Clustermatch (low)",
        # "Pearson (low)",
        # "Spearman (low)",
    ],
    [
        "Clustermatch (high)",
        "Pearson (high)",
        # "Spearman (high)",
        # "Clustermatch (low)",
        # "Pearson (low)",
        # "Spearman (low)",
    ],
    [
        "Clustermatch (high)",
        # "Pearson (high)",
        "Spearman (high)",
        # "Clustermatch (low)",
        # "Pearson (low)",
        # "Spearman (low)",
    ],
    [
        # "Clustermatch (high)",
        "Pearson (high)",
        "Spearman (high)",
        # "Clustermatch (low)",
        # "Pearson (low)",
        # "Spearman (low)",
    ],
    [
        # "Clustermatch (high)",
        # "Pearson (high)",
        # "Spearman (high)",
        "Clustermatch (low)",
        "Pearson (low)",
        # "Spearman (low)",
    ],
    [
        # "Clustermatch (high)",
        # "Pearson (high)",
        # "Spearman (high)",
        "Clustermatch (low)",
        # "Pearson (low)",
        "Spearman (low)",
    ],
    [
        # "Clustermatch (high)",
        # "Pearson (high)",
        # "Spearman (high)",
        # "Clustermatch (low)",
        "Pearson (low)",
        "Spearman (low)",
    ],
    [
        # "Clustermatch (high)",
        # "Pearson (high)",
        # "Spearman (high)",
        "Clustermatch (low)",
        "Pearson (low)",
        "Spearman (low)",
    ],
]

In [ ]:
display(len(_queries))
assert len(_queries) == 8

In [ ]:
gene_pairs_df = []

for idx, _query in enumerate(_queries):
    group_df = get_gene_pairs(gene_pairs_intersections, set(_query))
    gene_pairs_df.append(group_df)

gene_pairs_df = pd.concat(gene_pairs_df)

In [ ]:
display(gene_pairs_df.shape)
assert gene_pairs_df.drop_duplicates().shape == gene_pairs_df.shape
assert (gene_pairs_df.shape[0] > 7.3e6) and (gene_pairs_df.shape[0] < 7.4e6)

In [ ]:
gene_pairs_df.head()

# Create samples and save

In [ ]:
for sample_id in range(N_SAMPLES):
    data_sample = gene_pairs_df.sample(n=SAMPLE_SIZE, random_state=sample_id)

    output_filepath = OUTPUT_FILE_TEMPLATE.format(sample_id=sample_id)
    display(output_filepath)

    data_sample.to_pickle(output_filepath)

In [ ]:
display(data_sample.dtypes)
display(data_sample.shape)
display(data_sample.head())